In [1]:
import sys

sys.path.append("..")

In [2]:
from main import OCRModel
from inference import Inference

/home/qsvm/miniconda3/envs/new_ocr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/qsvm/miniconda3/envs/new_ocr/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [3]:
! huggingface-cli download ducto489/ocr_model viet.ckpt --repo-type model --local-dir checkpoints/pretrained_viet --local-dir-use-symlinks False

/home/qsvm/miniconda3/envs/new_ocr/lib/python3.11/site-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
checkpoints/pretrained_viet/viet.ckpt


In [4]:
checkpoint = "checkpoints/pretrained_viet/viet.ckpt"

In [18]:
inference = Inference(image_path="images") 
predict_dataloader = inference.predict_dataloader()
model = OCRModel.load_from_checkpoint(
    checkpoint, strict=True, batch_max_length=200, dali=True, map_location="cuda", pred_name="attn"
)
model.eval()

import time

start_time = time.time()
for batch in predict_dataloader:
    batch_start = time.time()
    preds = model.predict_step(batch, 0)
    batch_time = time.time() - batch_start
    print(f"Predictions: {preds}")
    print(f"Batch inference time: {batch_time:.3f}s")

total_time = time.time() - start_time
print(f"\nTotal inference time: {total_time:.3f}s")

2025-05-25 05:31:01.770 | INFO     | main:_build_model:68 - resnet18


backbone_name: resnet18
backbone_fac: {'resnet18': <class 'models.backbones.resnet18'>, 'resnet50': <class 'models.backbones.resnet50'>, 'vgg': <class 'models.backbones.VGG'>}


2025-05-25 05:31:02.607 | INFO     | main:__init__:65 - self.dali=True


Predictions: ['Chennai (Madras)']
Batch inference time: 0.082s
Predictions: ['because the sources of the pollution would be outside of urban areas']
Batch inference time: 0.093s
Predictions: ['CLEANED']
Batch inference time: 0.073s
Predictions: ['Chennai (Madras)']
Batch inference time: 0.074s
Predictions: ['because the sources of the pollution would be outside of urban areas']
Batch inference time: 0.074s
Predictions: ['CLEANED']
Batch inference time: 0.072s
Predictions: ['Chennai (Madras)']
Batch inference time: 0.074s
Predictions: ['because the sources of the pollution would be outside of urban areas']
Batch inference time: 0.075s
Predictions: ['CLEANED']
Batch inference time: 0.075s
Predictions: ['Chennai (Madras)']
Batch inference time: 0.076s
Predictions: ['because the sources of the pollution would be outside of urban areas']
Batch inference time: 0.074s
Predictions: ['CLEANED']
Batch inference time: 0.091s
Predictions: ['Chennai (Madras)']
Batch inference time: 0.075s
Predictio

KeyboardInterrupt: 

Test Inference time with VietOCR

In [9]:
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from PIL import Image

# config = Cfg.load_config_from_file('config.yml') # sử dụng config của các bạn được export lúc train nếu đã thay đổi tham số  
config = Cfg.load_config_from_name('vgg_transformer') # sử dụng config mặc định của mình 
# config['weights'] = '/home/qsvm/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth' # đường dẫn đến trọng số đã huấn luyện hoặc comment để sử dụng pretrained model của mình
config['device'] = 'cuda:0' # device chạy 'cuda:0', 'cuda:1', 'cpu'

detector = Predictor(config)

img = './images/1/paper_img_002844.png'
img = Image.open(img)
# dự đoán 
start_time = time.time()
s = detector.predict(img, return_prob=False) # muốn trả về xác suất của câu dự đoán thì đổi return_prob=True
inference_time = time.time() - start_time
print(f"VietOCR inference time: {inference_time:.3f}s")

Model weight /tmp/vgg_transformer.pth exsits. Ignore download!
VietOCR inference time: 0.565s


In [20]:
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from PIL import Image

# config = Cfg.load_config_from_file('config.yml') # sử dụng config của các bạn được export lúc train nếu đã thay đổi tham số  
config = Cfg.load_config_from_name('vgg_seq2seq') # sử dụng config mặc định của mình 
# config['weights'] = '/home/qsvm/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth' # đường dẫn đến trọng số đã huấn luyện hoặc comment để sử dụng pretrained model của mình
config['device'] = 'cuda:0' # device chạy 'cuda:0', 'cuda:1', 'cpu'

detector = Predictor(config)

img = './images/1/paper_img_002844.png'
img = Image.open(img)
# dự đoán 
start_time = time.time()
s = detector.predict(img, return_prob=False) # muốn trả về xác suất của câu dự đoán thì đổi return_prob=True
inference_time = time.time() - start_time
print(f"VietOCR inference time: {inference_time:.3f}s")

10935it [00:29, 376.59it/s]
/home/qsvm/miniconda3/envs/new_ocr/lib/python3.11/site-packages/vietocr/tool/predictor.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model

VietOCR inference time: 0.030s
